<a href="https://colab.research.google.com/github/nagamamo/financial_data_science/blob/main/8_1_nelson_siegel_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

8.1　パラメトリック・モデルによるイールドカーブの推計

　パラメトリック・モデルは、金利の期間構造を、それに影響を与える複数のパラメータを推計することで、イールドカーブの変動を説明するためのモデルです。もっともよく知られる手法が、米ワシントン大学（当時）のネルソン教授とシーゲル教授が開発したネルソン＝シーゲル・モデル（Nelson=Siegel Model）と呼ばれる手法です。ネルソン＝シーゲル・モデルでは、次のモデルを推計することで、金利の期間構造の変動を4つの要因に分解します。

r(m) = β0+(β1+β2 ){1-exp⁡(-m⁄τ) }m⁄τ
-β2 exp⁡(-m⁄τ)


　ここでは、β0：長期要因、β1：短期要因、β2：中期要因のパラメータ、m：償還期間、τ：構造要因のパラメータ

本例では、PyCurveによるこの実装事例を示します。

In [ ]:
pip install PyCurve

パッケージのインストールを行った後、PyCurveからネルソン＝シーゲル・モデル推計用のライブラリをnumpyとともにインポートします。

In [2]:
#[1]ライブラリの読み込み
from PyCurve.curve import Curve#PyCurveよりイールドカーブ分析ライブラリをインポート
from PyCurve.nelson_siegel import NelsonSiegel#PyCurveよりNSモデル推計ライブラリをインポート
import numpy as np

　続いてイールドカーブの償還期間（time）と利回り（d_rate）の2変数の初期値を設定します。本例では、償還期間は最大30年としています。rateは直近の実績値を用います。

In [3]:
#[2]データ初期値の設定
time = np.array([0.25,0.5,0.75,1.,2.,3.,4.,5.,10.,15.,20.,25.,30.])#償還期間
drate = np.array([-0.532,-0.530,-0.534,-0.516,-0.501,-0.498,-0.488,-0.313,-0.102,0.183,0.330,0.392,0.412])#直近の利回り
curve = Curve(time,drate)#2変数によるイールドカーブの構築
#curve1 = Curve(time,rate)#2変数によるイールドカーブの構築

　最後にパラメータの初期値β0、β1、β2、τを暫定的に与え、キャリブレートすることで、パラメータを推定します。

In [4]:
#[3]ネルソン＝シーゲル・モデルの推計
ns = NelsonSiegel(0.3,0.4,12,1)#パラメータの初期値を設定
ns.calibrate(curve)#NSモデルの推計
#ns.calibrate(curve1)#NSモデルの推計

Nelson Siegel Model
beta0 = 0.7520934553348047
beta1 = -1.2558210273712136
beta2 = -1.650443314839586
tau = 3.1635962850984387
____________________________
Calibration Results
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Mean Squared Error 0.012469766603992022
Number of Iterations 24
____________________________


  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.012469766603992022
        x: [ 7.521e-01 -1.256e+00 -1.650e+00  3.164e+00]
      nit: 24
      jac: [ 4.461e-06 -3.432e-06  8.842e-07 -1.311e-06]
     nfev: 135
     njev: 27
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>